In [1]:
# General Libraries
import numpy as np
from tdmclient import ClientAsync
import time
import matplotlib.pyplot as plt

# Custom libraries 
import filtering
import local_navigation as ln

client = ClientAsync()
node = await client.wait_for_node()
await node.lock()

Node 7648c768-a4cd-4427-9998-204136028638

In [2]:
#MAIN CODE : 

THRESHOLD_OBST = 3500 
command_motor = [102, 100]
speed_history = []
angular_speed_history = []
norm_distance = []
angle = []
local_obstacle = False

async def main():

    global local_obstacle
    start_time = time.time()
    state_estimation = np.array([[0], [0], [0], [0], [0]])
    P_estimation = 1000 * np.ones(5)
    bool_camera = False
    camera_position = np.array([[0], [0], [0]]) # get_camera_position()

    while(1):
        node = await client.wait_for_node()
        sens = await ln.get_proximity_values(node)

        if (sum(sens[i] > THRESHOLD_OBST for i in range(0, 5)) > 0): local_obstacle = True

        if(local_obstacle):
            node.send_set_variables(ln.motors(0,0))
            break

        if not local_obstacle: 
            node.send_set_variables(ln.motors(command_motor[0], command_motor[1]))
            state_estimation, P_estimation, speed, angular_speed, start_time = await filtering.get_position(state_estimation, P_estimation, start_time, bool_camera, camera_position, node)
            speed_history.append(speed)
            angular_speed_history.append(angular_speed)
            print("state_estimate angle", state_estimation[2][0]*180/np.pi, end="\r")
            norm_distance.append(np.sqrt(state_estimation[0][0]**2 + state_estimation[1][0]**2))
            print("distance", norm_distance[-1], end="\r")
            angle.append((state_estimation[2][0]*180)/np.pi)

        if (norm_distance[-1] > 300):
            node.send_set_variables(ln.motors(0,0))
            break

await main()

In [12]:
await node.set_variables(ln.motors(0, 0))

In [3]:
std_speed = np.std(speed_history[30:])
std_angular_speed = np.std(angular_speed_history[30:])
print("The speed variance in mm^2/s^2 is {}".format(std_speed))
print("The angular speed variance in rad^2/s^2 is {}".format(std_angular_speed))
print(len(speed_history))

plt.plot(norm_distance[:], label="distance")
plt.xlabel("Time step")
plt.ylabel("distance (mm)")
plt.legend()

plt.plot(angle[:], label="angle")
plt.xlabel("Time step")
plt.ylabel("angle (deg)")
plt.legend()

NameError: name 'speed_history' is not defined